In [59]:
import pandas as pd
import numpy as np

import os

In [60]:
PROCESSED_DIR = '../data/processed/'
PROCESSED_FILENAME = "usdjpy-bar-2020-01-01-2024-12-31_processed.pkl"
PROCESSED_FILE_PATH = os.path.join(PROCESSED_DIR, PROCESSED_FILENAME)

In [61]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,close_direction,prob_down,prob_flat,prob_up,label
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,1,-0.0095,-0.000087,down,1.0,0.0,0.0,0
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,1,0.0040,0.000037,up,0.0,0.0,1.0,2
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,1,0.0020,0.000018,flat,0.0,1.0,0.0,1
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,1,0.0095,0.000087,up,0.0,0.0,1.0,2
5,2020-01-01 22:05:00,108.7700,108.7700,108.7690,108.7700,1059.999987,1,0.0050,0.000046,up,0.0,0.0,1.0,2


In [62]:
df.describe()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,prob_down,prob_flat,prob_up,label
count,1818676,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06
mean,2022-07-05 13:20:55.965527552,1.281896e+02,1.282000e+02,1.281792e+02,1.281896e+02,4.311489e+05,4.203974e+03,3.065142e-05,2.461495e-07,3.382592e-01,3.180924e-01,3.436483e-01,1.005389e+00
min,2020-01-01 22:01:00,1.012020e+02,1.013380e+02,1.011820e+02,1.012045e+02,0.000000e+00,1.000000e+00,-2.065000e+00,-1.380804e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2021-04-01 18:33:45,1.094595e+02,1.094650e+02,1.094540e+02,1.094600e+02,1.180375e+05,1.847000e+03,-7.000000e-03,-5.705061e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2022-07-11 20:03:30,1.312150e+02,1.312340e+02,1.311990e+02,1.312150e+02,2.726000e+05,4.956000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,2023-10-04 07:22:15,1.456610e+02,1.456740e+02,1.456491e+02,1.456615e+02,5.858400e+05,6.143000e+03,7.500000e-03,5.846136e-05,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
max,2024-12-30 23:59:00,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,1.226455e+08,7.375000e+03,1.579500e+00,1.228753e-02,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
std,NaN,1.839441e+01,1.839714e+01,1.839161e+01,1.839442e+01,5.050698e+05,2.341641e+03,2.094446e-02,1.558500e-04,4.731174e-01,4.657357e-01,4.749255e-01,8.257596e-01


In [63]:
GROUP_COL = 'time_group'
SEQ_LEN = 30
HORIZON = 1

In [64]:
def get_sequence_start_indices(df, sequence_length=30, horizon=1, stride=1, group_col='time_group'):
    indices = []
    group_to_indices = {}
    
    for idx, group in zip(df.index, df[group_col]):
        group_to_indices.setdefault(group, []).append(idx)

    for group, idxs in group_to_indices.items():
        if len(idxs) < sequence_length + horizon:
            continue

        idxs = sorted(idxs)
        max_start = len(idxs) - sequence_length - horizon + 1
        for start in range(0, max_start, stride):
            indices.append(idxs[start])

    return indices


In [65]:
IDs = get_sequence_start_indices(
    df, 
    sequence_length=30,
    horizon=1,
    stride=5,
    group_col='time_group',
)

In [66]:
df.loc[IDs[-1]]

timestamp          2024-12-30 23:26:00
open                            157.03
high                           157.032
low                             157.03
close                           157.03
volume                    36450.000405
time_group                        7375
close_delta                    -0.0015
close_return                  -0.00001
close_direction                   flat
prob_down                          0.0
prob_flat                          1.0
prob_up                            0.0
label                                1
Name: 1860606, dtype: object

In [138]:
import torch
from torch.utils.data import Dataset
import numpy as np

class ForexDataset(Dataset):
    """Dataset for sequence classification/forecasting with multi-step horizon."""

    def __init__(self, data, IDs, sequence_length, horizon, features, target):
        self.data = data
        self.IDs = IDs
        self.sequence_length = sequence_length
        self.horizon = horizon
        self.feature_data = data[features]
        self.target_data = data[target]
    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, idx):
        i = self.IDs[idx]
        
        # Extract feature sequence
        X = self.feature_data.loc[i:i + self.sequence_length].values

        # Extract target(s)
        y = self.target_data.loc[i+self.sequence_length + self.horizon - 1].values

        return torch.from_numpy(X), torch.from_numpy(y), i


In [139]:
fx_dataset = ForexDataset(
    data=df, 
    IDs=IDs,
    sequence_length=SEQ_LEN, 
    horizon=HORIZON, 
    features=['close_return'],
    target=['close_return'],
)
    

In [151]:
fx_dataset.feature_data.loc[31]--4.5993e-06

close_return    1.288950e-11
Name: 31, dtype: float64

In [144]:
fx_dataset[0]

(tensor([[-8.7349e-05],
         [ 3.6782e-05],
         [ 1.8390e-05],
         [ 8.7352e-05],
         [ 4.5971e-05],
         [-1.1952e-04],
         [ 4.5974e-06],
         [-2.7584e-05],
         [-7.8158e-05],
         [-9.1957e-06],
         [-4.5979e-05],
         [ 1.6093e-04],
         [-9.1948e-05],
         [-2.2989e-05],
         [ 2.2990e-05],
         [-4.5978e-06],
         [-9.1957e-06],
         [-8.6440e-04],
         [ 3.5894e-04],
         [ 1.3801e-05],
         [ 5.0601e-05],
         [-4.5999e-06],
         [ 1.6560e-04],
         [ 5.0591e-05],
         [-1.0578e-04],
         [ 4.5994e-06],
         [-4.5994e-06],
         [ 1.7478e-04],
         [-1.4256e-04],
         [-4.5993e-06],
         [-4.5993e-06]], dtype=torch.float64),
 tensor([-4.5993e-06], dtype=torch.float64),
 1)

In [135]:
df.loc[51:82, ['close_return']]

,close_return
51,-0.000009
52,0.000000
53,0.000005
54,-0.000005
55,-0.000009
56,0.000023
57,-0.000005
58,0.000014
59,0.000005
60,-0.000570


In [105]:
fx_dataset = ForexClassificationDataset(
    data=df, 
    IDs=IDs,
    sequence_length=SEQ_LEN, 
    horizon=1, 
    features=['close_return'],
    target=['close_return'],
)
    

In [106]:
fx_dataset[5]

(tensor([[ 4.5994e-06],
         [-4.5994e-06],
         [ 1.7478e-04],
         [-1.4256e-04],
         [-4.5993e-06],
         [-4.5993e-06],
         [-3.2195e-05],
         [ 0.0000e+00],
         [ 9.1989e-06],
         [-1.3798e-04],
         [-1.2420e-04],
         [ 4.6006e-06],
         [ 4.6006e-06],
         [-1.3802e-05],
         [ 1.3802e-05],
         [-2.3003e-05],
         [ 1.8403e-05],
         [-4.6006e-06],
         [ 9.2012e-06],
         [ 3.2204e-05],
         [ 1.8402e-05],
         [ 0.0000e+00],
         [ 9.2007e-06],
         [ 0.0000e+00],
         [ 1.8401e-05],
         [-9.2004e-06],
         [ 0.0000e+00],
         [ 4.6003e-06],
         [-4.6002e-06],
         [-9.2005e-06]]),
 tensor([2.3002e-05]),
 26)

## Good old train / test split

In [79]:
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(
    IDs,
    test_size=0.2,
    shuffle=True,
    random_state=42
)


In [80]:
train_dataset = ForexClassificationDataset(
    data=df, 
    IDs=train_idx,
    sequence_length=SEQ_LEN, 
    horizon=HORIZON, 
    features=['close_return'],
    target=['prob_down', 'prob_flat', 'prob_up'],
)

val_dataset = ForexClassificationDataset(
    data=df, 
    IDs=val_idx,
    sequence_length=SEQ_LEN, 
    horizon=HORIZON, 
    features=['close_return'],
    target=['prob_down', 'prob_flat', 'prob_up'],
)

In [81]:
len(train_dataset), len(val_dataset)

(278025, 69507)

In [85]:
train_dataset[-1]

(tensor([[-3.9572e-05],
         [-1.7588e-04],
         [ 4.8376e-05],
         [-8.7952e-06],
         [ 4.3976e-06],
         [ 7.9157e-05],
         [-8.3548e-05],
         [ 4.3976e-06],
         [ 4.8374e-05],
         [-4.3974e-06],
         [ 7.0358e-05],
         [-1.1432e-04],
         [ 4.3976e-05],
         [-8.7948e-06],
         [-3.9577e-05],
         [ 1.8470e-04],
         [-4.3968e-05],
         [-3.9573e-05],
         [ 1.0993e-04],
         [-7.9140e-05],
         [ 3.5176e-05],
         [-7.9144e-05],
         [ 5.7164e-05],
         [-7.0352e-05],
         [ 5.7165e-05],
         [ 1.4071e-04],
         [ 8.7928e-06],
         [-4.3964e-05],
         [ 3.5173e-05],
         [ 0.0000e+00]]),
 tensor([[0., 0., 1.]]),
 678787)

## Stratified Shuffle

In [ ]:
valid_start_idxs[:5]

In [ ]:
target_indices = np.array(valid_start_idxs) + SEQ_LEN + HORIZON - 1


In [ ]:
target_indices

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

def get_stratified_split(df, valid_start_indices, sequence_length, horizon, target_col, test_size=0.2):
    valid_start_indices = np.array(valid_start_indices)  # ensure array

    target_indices = valid_start_indices + sequence_length + horizon - 1
    labels = df.loc[target_indices, target_col].values  # get labels efficiently

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_idx, val_idx = next(splitter.split(valid_start_indices, labels))

    train_start_indices = valid_start_indices[train_idx]
    val_start_indices = valid_start_indices[val_idx]

    return train_start_indices.tolist(), val_start_indices.tolist()


In [ ]:
train_idx, test_idx = get_stratified_split(
    df,
    valid_start_indices=valid_start_idxs,
    sequence_length=SEQ_LEN,
    horizon=HORIZON,
    target_col='label'
)

In [ ]:
len(train_idx), len(test_idx)